In [0]:
import numpy as np
from zipfile import ZipFile
from os import listdir
from os.path import isfile, join
import cv2
import scipy.io
from matplotlib import pyplot as plt 
from PIL import Image
from sklearn import preprocessing
from sklearn import cluster
from sklearn import metrics
from sklearn.metrics.cluster import v_measure_score
from sklearn.cluster import SpectralClustering

In [0]:
# unzipping
zip=ZipFile('BSR.zip')
zip.extractall()
zip.close()

OSError: ignored

In [0]:
# function reads matlab file and displays the groundtruth segmentations
segmentations=[]
def read_GroundTruth(img_path,gt_path):
        #segmentations=[]
        #img=plt.imread(img_path)                                          #reading image from img-path
        #imgList.append(img)                                               #to create array of images
        mat = scipy.io.loadmat(gt_path,appendmat=False)                     #load matlab file
        groundTruth = mat.get('groundTruth')                                #get image
        label_num = groundTruth.size
        #print(groundTruth)
        
        #converting the matlab arrays to images
        fig=plt.figure(figsize=(10,10))
        for i in range(label_num):
            boundary = groundTruth[0][i]['Segmentation'][0][0]
            height = boundary.shape[0]
            width = boundary.shape[1]
            trueBoundary = boundary.reshape(height, width, 1)
            segmentations.append(trueBoundary)
            fig.add_subplot(5,5, i+1)
            plt.imshow(trueBoundary[:,:,0])

In [0]:
#function that convert images to its RBG representation and apply Kmeans for clustering 
K_segmentations=[]
def KMEANS(img):
#convert image to 2D with M*3 dimensions
        #img=np.array(img)
        x, y, z = img.shape
        image_2d = img.reshape(x*y, z)
        image_2d.shape
        
        #Apply K-means on the 2D image
        kmeans_cluster = cluster.KMeans(n_clusters=5)
        kmeans_cluster.fit(image_2d)
        cluster_centers = kmeans_cluster.cluster_centers_
        cluster_labels = kmeans_cluster.labels_
        print("de labels beta3et el kmeans")
        print(cluster_labels)
        K_segmentations.append(cluster_labels)
    #Convert back into uint8 and make an image
        centers=np.array(cluster_centers)
        labels=np.transpose(np.uint8(cluster_labels))
        res=centers[labels]
        res=res.reshape(x,y,z)
        fig2 = plt.figure(3)
        plt.imshow(res[:,:,0])
        plt.show()



In [0]:
#function to evaluate segmentations using F-measures and conditional Entropy
def evaluate(boundary,pred_labels):
    print(v_measure_score(boundary,pred_labels))

In [0]:
#normalized cut ussing 5-NN graph
def Normalized_cut(img):
        x, y, z = img.shape
        image_2d = img.reshape(x*y, z)
        image_2d.shape
    
        Norm_cut=SpectralClustering(n_clusters=5, eigen_solver=None, random_state=0,affinity='nearest_neighbors', n_neighbors=5).fit(image_2d)

In [0]:
#directories
train_dir="/Users/geek/Desktop/BSR/BSDS500/data/images/train"
train_gt_dir= "/Users/geek/Desktop/BSR/BSDS500/data/groundTruth/train"
test_dir="/Users/geek/Desktop/BSR/BSDS500/data/images/test"
test_gt_dir= "/Users/geek/Desktop/BSR/BSDS500/data/groundTruth/test"

#generating images ang gt images directories
img_files = [f for f in listdir(test_dir) if isfile(join(test_dir, f)) and f.endswith(".jpg")]
gt_files = [f for f in listdir(test_gt_dir) if isfile(join(test_gt_dir, f)) and f.endswith(".mat")]

#global variables
segmentations=[]
K_segmentations=[]
#groundTruth_labels=[]

#reading images and groundTruth

for i in range(50):
        print("Handling:   " + gt_files[i] + "   ====   " + img_files[i])
        img_path = "/Users/geek/Desktop/BSR/BSDS500/data/images/test/"+img_files[i]
        gt_path = "/Users/geek/Desktop/BSR/BSDS500/data/groundTruth/test/" + gt_files[i]
        img=plt.imread(img_path)                                          #reading image from img-path
        #imgList.append(img) 
        fig1=plt.figure(1)
        plt.imshow(img)
        plt.show()
        read_GroundTruth(img_path,gt_path)     #to display  ground truth
        #ba fasfafhaaa
        s=np.array(segmentations)
        n=s.shape
        noOfsegmentations=n[0]
        x=n[1]
        y=n[2]
        z=n[3]
         #x, y, z = img.shape
        #image_2d = img.reshape(x*y, z)
        #image_2d.shape
        print(n)
        fig=plt.figure(figsize=(10,10))
       # KMEANS(img)                           #for clustering
        Normalized_cut(img)
        pred_labels=np.array(K_segmentations)
        pred_labels=pred_labels.flatten()
        print(pred_labels.shape)
        #converting the groundTruth segmentations to 1d labels to perform the evaluations
        for j in range(noOfsegmentations):
            boundary = s[j].reshape(x*y,z)
            boundary=boundary.flatten()
            evaluate(boundary,pred_labels)    # evaluation
            print("de labels beta3et el ground truth")
            print(boundary)
            #print(trueBoundary.shape)
        
        
        break



FileNotFoundError: ignored